# Weekend Movie Trip

In [67]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [68]:
dfLinks=pd.read_csv("/Users/anissakhan/Dropbox/Nissa_stuff/KU_CS/EECS731/731-Project-3-Khan/Data/RawData/ml-latest-small/links.csv")
dfLinks.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [69]:
dfMovies=pd.read_csv("/Users/anissakhan/Dropbox/Nissa_stuff/KU_CS/EECS731/731-Project-3-Khan/Data/RawData/ml-latest-small/movies.csv")
dfMovies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [70]:
dfRatings=pd.read_csv("/Users/anissakhan/Dropbox/Nissa_stuff/KU_CS/EECS731/731-Project-3-Khan/Data/RawData/ml-latest-small/ratings.csv")
dfRatings["timestampRating"]=dfRatings["timestamp"]
dfRatings.drop(columns =["timestamp"], inplace = True) 
dfRatings.head()

,userId,movieId,rating,timestampRating
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [71]:
dfTags=pd.read_csv("/Users/anissakhan/Dropbox/Nissa_stuff/KU_CS/EECS731/731-Project-3-Khan/Data/RawData/ml-latest-small/tags.csv")
dfTags["timestampTag"]=dfTags["timestamp"]
dfTags.drop(columns =["timestamp"], inplace = True) 
dfTags.head()

,userId,movieId,tag,timestampTag
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [72]:
dfRatings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   userId           100836 non-null  int64  
 1   movieId          100836 non-null  int64  
 2   rating           100836 non-null  float64
 3   timestampRating  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [73]:
dfTags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   userId        3683 non-null   int64 
 1   movieId       3683 non-null   int64 
 2   tag           3683 non-null   object
 3   timestampTag  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


In [74]:
new = dfMovies["title"].str.split("(", n = 1, expand = True)
dfMovies["titleOnly"]=new[0]
dfMovies["year"]=new[1]
dfMovies.head()

,movieId,title,genres,titleOnly,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995)
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji,1995)
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995)
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale,1995)
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II,1995)


In [75]:
dfMovies.dtypes

movieId       int64
title        object
genres       object
titleOnly    object
year         object
dtype: object

In [76]:
dfMovies['year'] = dfMovies['year'].astype("string")
dfMovies['titleOnly'] = dfMovies['titleOnly'].astype("string")
dfMovies['title'] = dfMovies['title'].astype("string")
dfMovies['genres'] = dfMovies['genres'].astype("string")
dfMovies.dtypes

movieId       int64
title        string
genres       string
titleOnly    string
year         string
dtype: object

In [77]:
new = dfMovies["year"].str.split(")", n = 1, expand = True)
dfMovies["year"]=new[0]
dfMovies.head()

,movieId,title,genres,titleOnly,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II,1995


In [78]:
dfAll=pd.merge(dfRatings, dfTags, left_on=['userId', 'movieId'], right_on=['userId', 'movieId'])
dfAll

,userId,movieId,rating,timestampRating,tag,timestampTag
0,2,60756,5.0,1445714980,funny,1445714994
1,2,60756,5.0,1445714980,Highly quotable,1445714996
2,2,60756,5.0,1445714980,will ferrell,1445714992
3,2,89774,5.0,1445715189,Boxing story,1445715207
4,2,89774,5.0,1445715189,MMA,1445715200
...,...,...,...,...,...,...
3471,606,6107,4.0,1171324428,World War II,1178473747
3472,606,7382,4.5,1171233924,for katie,1171234019
3473,610,3265,5.0,1479542010,gun fu,1493843984
3474,610,3265,5.0,1479542010,heroic bloodshed,1493843978


In [79]:
dfAll=pd.merge(dfAll, dfMovies, left_on=['movieId'], right_on=['movieId'])
dfAll

,userId,movieId,rating,timestampRating,tag,timestampTag,title,genres,titleOnly,year
0,2,60756,5.0,1445714980,funny,1445714994,Step Brothers (2008),Comedy,Step Brothers,2008
1,2,60756,5.0,1445714980,Highly quotable,1445714996,Step Brothers (2008),Comedy,Step Brothers,2008
2,2,60756,5.0,1445714980,will ferrell,1445714992,Step Brothers (2008),Comedy,Step Brothers,2008
3,62,60756,3.5,1528934376,comedy,1528934384,Step Brothers (2008),Comedy,Step Brothers,2008
4,62,60756,3.5,1528934376,funny,1528934381,Step Brothers (2008),Comedy,Step Brothers,2008
...,...,...,...,...,...,...,...,...,...,...
3471,606,1948,3.5,1177512639,British,1177512649,Tom Jones (1963),Adventure|Comedy|Romance,Tom Jones,1963
3472,606,5694,3.0,1172968972,70mm,1175638092,Staying Alive (1983),Comedy|Drama|Musical,Staying Alive,1983
3473,606,6107,4.0,1171324428,World War II,1178473747,Night of the Shooting Stars (Notte di San Lore...,Drama|War,Night of the Shooting Stars,"Notte di San Lorenzo, La"
3474,610,3265,5.0,1479542010,gun fu,1493843984,Hard-Boiled (Lat sau san taam) (1992),Action|Crime|Drama|Thriller,Hard-Boiled,Lat sau san taam


In [80]:
dfAll.dtypes

userId               int64
movieId              int64
rating             float64
timestampRating      int64
tag                 object
timestampTag         int64
title               string
genres              string
titleOnly           string
year                string
dtype: object